# Analysis of DIC data from tensile testing

## Import libraries

In [ ]:
import glob

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from ipywidgets import interact, fixed, IntSlider, FloatSlider
from dic_analysis import DeformationMap

%matplotlib inline 
%load_ext autoreload
%autoreload 2

## Analysis set up
Change the data location and number of files to load here.

In [ ]:
data_folder = "../Data/test 0-1/displacement data/"

Load data from files

In [ ]:
# Load data from files
file_list = glob.glob(f"{data_folder}*")
deformation_maps = [DeformationMap(file_path) for file_path in tqdm(file_list)]

## Plotting strain maps

This cell allows scanning through the strain maps over time. This can be used to determine which timesteps are interesting to output.

In [ ]:
def scrub_strain(measurement: int, maps):
    plt.imshow(maps[measurement].f22)
    plt.colorbar()

interact(scrub_strain, 
         measurement=IntSlider(min=1, max=np.size(deformation_maps) - 1, step=1, continuous_update=False),
         fig=fixed(plt.figure()), maps=fixed(deformation_maps), continuous_update=False)

Once the interesting maps have been identified, this cell allows saving any interesting strain maps to file.

In [ ]:
timesteps = [10, 50]

for step in timesteps:
    fig = plt.figure()

    # Display the image
    plt.imshow(deformation_maps[step].f22, cmap='viridis', interpolation='none')

    # Turn off axes and set axes limits
    plt.title(f"Strain map at timestep {step}", pad=10)
    plt.colorbar()
    fig.savefig(f"../results/strain_map_t_{step}.png", bbox_inches ="tight", dpi=200)
    plt.close()
print(f"Timesteps {timesteps} written to file.")

## Plotting shape change of sample

rho is the shape change -deyy/dexx

As before the first cell allows scrubbing through the dataset and the next cell allows saving of interesting timesteps.

In [ ]:
def scrub_rho(measurement: int, maps):
    rho = -maps[measurement].f11 / maps[measurement].f22
    plt.imshow(rho)
    plt.colorbar()

interact(scrub_rho, 
         measurement=IntSlider(min=1, max=np.size(deformation_maps) - 1, step=1, continuous_update=False),
         fig=fixed(plt.figure()), maps=fixed(deformation_maps), continuous_update=False)

In [ ]:
timesteps = [10, 50]

for step in timesteps:
    fig = plt.figure()
    def_map = deformation_maps[step]
    rho = -def_map.f11 / def_map.f22

    plt.imshow(rho, cmap='viridis', interpolation='none')
    plt.title(f"Shape change of sample at timestep {step}", pad=10)
    plt.colorbar()
    plt.savefig(f"../results/shape_change_t_{step}.png", bbox_inches ="tight", dpi=200)
    plt.close()

print(f"Timesteps {timesteps} written to file.")

## Plotting sample strain/true strain over time

In [ ]:
strain_list = []
for def_map in deformation_maps:
    cropped_map = def_map.f22[10:24, 1:12]
    strain_list.append(np.mean(cropped_map))
strain_list = np.array(strain_list)
    
plt.figure()
plt.plot(strain_list, label="strain")
plt.plot(np.log(1 + np.array(strain_list)), label="true strain")
plt.xlabel("Time step")
plt.ylabel("Strain")
plt.legend()
plt.show()

## Plotting ?? Over time

In [ ]:
trans_list = []
for def_map in deformation_maps:
    cropped_map = def_map.f11[10:24,1:12]
    
    trans_list.append(np.mean(cropped_map))
    
plt.figure()
plt.plot(trans_list)
plt.plot(strain_list)
plt.xlabel('Data Point')
plt.ylabel('Strain')
plt.savefig(f"../results/strain.png", bbox_inches ="tight", dpi=200)

## Plotting Strain ratio and Lankford Parameter

We crop the data at a max and min strain to avoid noisy data points

In [ ]:
strain_ratio = - np.array(trans_list) / np.array(strain_list)
Lankford = strain_ratio / (1 - strain_ratio)
plt.figure()
min_strain = 0.02
max_strain = 0.29
mask = np.logical_and(min_strain < strain_list, strain_list < max_strain)
plt.plot(strain_list[mask], strain_ratio[mask])
plt.xlabel("strain")
plt.ylabel("strain ratio")
plt.savefig(f"../results/strain_ratio.png", bbox_inches ="tight", dpi=200)

## Plotting Lankford parameter
As above we cut the data at a minimum and maximum strain to reduce noise.

In [ ]:
plt.figure()
plt.plot(strain_list[mask], Lankford[mask])
plt.xlabel("strain")
plt.ylabel("Lankford parameter")
plt.savefig(f"../results/lankford_parameter.png", bbox_inches ="tight", dpi=200)

## Plotting Measured strain data

In [ ]:
test_names = ["0-1", "0-2", "30-1", "30-2", "45-1", "45-2", "60-1", "60-2", "90-1", "90-2"]

plt.figure()

for name in test_names:
    voltage_data = np.loadtxt(f"../Data/test {name}/voltage data/data_1.csv", delimiter=",", skiprows=2, usecols=(4, 15))
    
    # Cut off data when it begins dropping at the end of the experiment
    data_limit = voltage_data.shape[0]
    for i in range(0, data_limit - 50):
        if voltage_data[i, 1] > voltage_data[i + 50, 1]:
            data_limit = i + 50
            break
            
    # Plot the data
    plt.plot(voltage_data[:data_limit , 0], voltage_data[:data_limit, 1], "x-", label=name)
    plt.xlabel("True Strain")
    plt.ylabel("True Stress (MPa)")
    plt.legend()
plt.show()